In [2]:
# Import dependencies
import pandas as pd
import numpy as np
import json 
import sqlite3
pd.set_option('max_colwidth', 400)

In [ ]:
# Read the data into a Pandas DataFrame
ship_wreck_info_df1 = pd.read_excel('AWOIS_Wrecks.xls')
ship_wreck_info_df1.head()

In [ ]:
# remove the columns that are not needed
ship_wreck_info_df1 = ship_wreck_info_df1.drop(columns=['RECRD', 'DEPTH', 'SOUNDING_TYPE'], axis=1)
# rename the columns
ship_wreck_info_df1 = ship_wreck_info_df1.rename(columns={'VESSLTERMS': 'name', 'FEATURE_TYPE': 'type', 'LATDEC': 'lat', 'LONDEC': 'lng', 'GP_QUALITY': 'accuracy', 'YEARSUNK': 'year_sunk', 'HISTORY': 'history'})
ship_wreck_info_df1.head()
# fill the null values of the year_sunk column with 0
ship_wreck_info_df1['year_sunk'] = ship_wreck_info_df1['year_sunk'].fillna(0)
# convert the year_sunk column to integer
ship_wreck_info_df1['year_sunk'] = ship_wreck_info_df1['year_sunk'].astype(int)
# drop the rows with nan values
ship_wreck_info_clean = ship_wreck_info_df1.dropna()
# add an id column
ship_wreck_info_clean['id'] = ship_wreck_info_clean.index + 1
# reorder the columns
ship_wreck_info_clean = ship_wreck_info_clean[['id', 'name', 'type', 'lat', 'lng', 'accuracy', 'year_sunk', 'history']]

In [ ]:
# find the longest number of characters in the history column
ship_wreck_info_clean['history'].str.len().max()

In [ ]:
ship_wreck_info_clean['accuracy'].unique()

In [ ]:
# save the cleaned data to a csv file
ship_wreck_info_clean.to_csv('shipwreck.csv', index=False)
# save the cleaned data to a json file
ship_wreck_info_clean.to_json('shipwreck.json', orient='records')

In [3]:
# create database from the shipwreck.csv file
conn = sqlite3.connect('shipwreck.sqlite')
c = conn.cursor()



In [12]:
# create a table with id as primary key
c.execute('''CREATE TABLE wrecks(id INTEGER PRIMARY KEY DESC, name TEXT, type TEXT, lat REAL, lng REAL, accuracy TEXT, year_sunk INTEGER, history TEXT)''')

In [13]:
# add csv data to the database
wrecks = pd.read_csv('shipwreck.csv')
# add the data to the database
wrecks.to_sql('wrecks', conn, if_exists='append', index=False)

In [8]:
# remove the wrecks table
c.execute('''DROP TABLE wrecks''')

In [4]:
# close the connection
conn.close()